In [8]:
import joblib
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn

class BestModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.gru_X = nn.GRU(
            input_size=7 * 24,
            hidden_size=hidden_size,
            num_layers=num_layers,
        )
        self.linear = nn.Linear(hidden_size, 24)

    def forward(self, x, meta):
        x, _ = self.gru_X(x)
        x = self.linear(x)
        return x

X_scaler = joblib.load("X_scaler.joblib")
model = torch.load("torch_LSTM.pickle")

from load_data import load_data, split_data

sampling = "1H"

data = load_data(sampling)

In [9]:
columns_to_drop = [
    "CH_AT",
    "CH_DE",
    "CH_FR",
    "CH_IT",
    "AT_CH",
    "DE_CH",
    "FR_CH",
    "IT_CH",
]

In [10]:
data = data.drop(columns=columns_to_drop)

In [11]:
X, metadata, y = split_data(data, 7 * 24, 24, 1, "MWh")

X.shape, y.shape

((26112, 168), (26112, 24))

In [12]:
X = X_scaler.transform(X)

In [13]:
X = torch.Tensor(X)[:, :].to(device='cuda')
y = torch.Tensor(y)[:, :].to(device='cuda')
metadata = torch.Tensor(metadata).to(device='cuda')

In [15]:
## import torch.optim as optim
from torch import utils

loader = utils.data.DataLoader(
    utils.data.TensorDataset(X, metadata, y),
    shuffle=False,
    batch_size=1024,
)
    
model.eval()
with torch.no_grad():
    y_pred = model(X, metadata)
    mae = nn.L1Loss()(y_pred, y).item()
print("MAE %.4f" % (mae))

MAE 16.4330


In [ ]:
torch.save(model, "tor")